In [0]:
%config InlineBackend.figure_formats = ['svg']
import sys
sys.path.insert(1, '../')

import matplotlib.pyplot as plt
from os import path

from deployment_instance.Result import ExperimentResult
from src.ingest_data import ingest_experiment_results
from src.query_data import get_exfiltration_time_df, get_data_exfiltration_cdf, total_control_host_capture_times
from statistics import mean
import seaborn as sns
# set seaborn font size
sns.set_theme(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid' : False})
sns.color_palette("pastel")

import pandas as pd

In [6]:
# Equifax 
equifax_reactiveSimple_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/equifax_reactiveSimple'))
equifax_advancedReactive_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/baseline_reactiveStandalone_d10_h0'))
equifax_advancedReactive_d0_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/baseline_reactiveStandalone_d0_h50'))
equifax_advancedReactive_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/baseline_reactiveStandalone_d10_h50'))
equifax_advancedReactive_layered_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/baseline_reactiveLayered_d10_h50'))


In [2]:
equifax_reactiveSimple_d10_h0_times = get_exfiltration_time_df(equifax_reactiveSimple_d10_h0, 48, 'targeted_reactiveSimple_d10_h0')
equifax_advancedReactive_d10_h0_times = get_exfiltration_time_df(equifax_advancedReactive_d10_h0, 48, 'targeted_advancedReactive_d10_h0')
equifax_advancedReactive_d0_h50_times = get_exfiltration_time_df(equifax_advancedReactive_d0_h50, 48, 'targeted_advancedReactive_d0_h50')
equifax_advancedReactive_d10_h50_times = get_exfiltration_time_df(equifax_advancedReactive_d10_h50, 48, 'targeted_advancedReactive_d10_h50')
equifax_advancedReactive_layered_d10_h50_times = get_exfiltration_time_df(equifax_advancedReactive_layered_d10_h50, 48, 'targeted_advancedReactive_d10_h50')

# Merge dfs into 1 df
equifax_data = pd.concat([equifax_reactiveSimple_d10_h0_times,
                          equifax_advancedReactive_d10_h0_times, 
                          equifax_advancedReactive_d0_h50_times, 
                          equifax_advancedReactive_d10_h50_times,
                          equifax_advancedReactive_layered_d10_h50_times])

FileNotFoundError: [Errno 2] No such file or directory: 'data/EquifaxLarge/reactive/equifax/baseline_reactiveSimple_d10_h0'

In [3]:
equifax_data_agg = (
    equifax_data.groupby('experiment')
    .agg({'percent_files_exfiltrated': {'mean'}, 'time_exfiltrated': 'median', 'defender': 'first', 'attacker': 'first'})
    .reset_index()
)
equifax_data_agg

NameError: name 'equifax_data' is not defined

In [4]:
# Box plot
plt.figure(figsize=(10, 6))
sns.boxplot(data=equifax_data, x='experiment', y='percent_files_exfiltrated',
            hue='experiment', palette='pastel', showfliers=False)

NameError: name 'equifax_data' is not defined

<Figure size 1000x600 with 0 Axes>